In [1]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
from smount_predictors import SeamountTransformer
from smount_predictors import SeamountHelp
import plotly.express as px
import pandas as pd

In [2]:
coords = SeamountHelp.readKMLbounds(Path('data') / 'Seamount_training_zone.kml')
training_data = SeamountHelp.readAndFilterGRD(Path('data') / 'vgg_swot.grd', coords[:2], coords[2:])

In [3]:
tran_inst = SeamountTransformer()
tran_inst.sig = 0.5

In [4]:
numpy_train = training_data.to_dataframe().reset_index()[['lon', 'lat', 'z']].to_numpy()

In [5]:
trans = tran_inst.transform(numpy_train)

In [6]:
trans_df = pd.DataFrame(trans, columns=['lon', 'lat', 'z'])
px.scatter(trans_df, x='lon', y='lat', color='z', title='Transformed data').show()